In [ ]:
from astropy.coordinates import SkyCoord, AltAz
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import EarthLocation
import pandas as pd
from datetime import datetime

def convert_deg_to_hms(deg):
    """Convierte grados a formato horas, minutos y segundos de manera precisa."""
    # Asegurarse de que el ángulo esté en el rango [0, 360)
    deg = deg % 360  
    hours_total = deg / 15  # 1 hora = 15 grados
    hours = int(hours_total)
    minutes_total = (hours_total - hours) * 60
    minutes = int(minutes_total)
    seconds = (minutes_total - minutes) * 60
    
    # Asegurarse de que los segundos no excedan 60
    if seconds >= 60:
        seconds -= 60
        minutes += 1
    if minutes >= 60:
        minutes -= 60
        hours += 1
    
    return f"{hours:02}h{minutes:02}m{seconds:.2f}s"

def convert_deg_to_gms(deg):
    """Convierte grados a formato grados, minutos y segundos con signo correcto."""
    sign = "-" if deg < 0 else ""
    deg = abs(deg)
    degrees = int(deg)
    minutes_full = (deg - degrees) * 60
    minutes = int(minutes_full)
    seconds = (minutes_full - minutes) * 60
    
    # Ajustar desbordamientos (segundos >= 60 o minutos >= 60)
    if seconds >= 60:
        seconds -= 60
        minutes += 1
    if minutes >= 60:
        minutes -= 60
        degrees += 1
    
    # Formatear grados con signo, pero sin relleno de cero si es negativo
    degrees_str = f"{sign}{abs(degrees)}"  # Ej: "-45" o "30"
    
    return f"{degrees_str}°{minutes:02}m{seconds:.2f}s"
# Cargar la tabla
tabla = pd.read_csv("Listado de Videos - Lista2.csv")

# Definir tu ubicación (ejemplo)
location = EarthLocation(lat=3.23465 * u.deg, lon=- 75.17 * u.deg, height=459 * u.m) # P1


# Listas para guardar resultados
ra_list = []
dec_list = []
ha_list = []  # Lista para almacenar el ángulo horario

for index, row in tabla.iterrows():
    fecha = row["Fecha"]
    hora_ut = row["Hora_UT"]
    estrella = row["Estrella"]
    print(f"Procesando fila {index}: {fecha}, {hora_ut}, {estrella}")

    # Limpiar espacios
    fecha = fecha.strip()
    hora_ut = hora_ut.strip()
    estrella = estrella.strip()

    # Convertir fecha
    fecha_dt = datetime.strptime(fecha, "%m/%d/%Y")
    fecha_formateada = fecha_dt.strftime("%Y-%m-%d")

    time_obs = Time(f"{fecha_formateada}T{hora_ut}", scale="utc", location=location)

    # Obtener las coordenadas de la estrella
    coord = SkyCoord.from_name(estrella)

    # Calcular el tiempo sideral local (LST)
    lst = time_obs.sidereal_time("apparent")  # Tiempo sideral local

    # Calcular el ángulo horario (HA)
    ha = (lst - coord.ra).wrap_at(360 * u.deg)

    # Guardar resultados
    ra_list.append(convert_deg_to_hms(coord.ra.deg))
    dec_list.append(convert_deg_to_gms(coord.dec.deg))
    ha_list.append(ha)

# Agregar resultados a la tabla
tabla["RA_deg"] = ra_list
tabla["Dec_deg"] = dec_list
tabla["HA_deg"] = ha_list  # Agregar columna de ángulo horario

# Guardar nueva tabla
tabla.to_csv("tabla_salida_con_HA.csv", index=False)

Procesando fila 0: 11/6/2018, 23:53:41, Altair
Procesando fila 1: 11/6/2018, 23:54:57, Altair
Procesando fila 2: 11/6/2018, 23:56:47, Altair
Procesando fila 3: 11/7/2018, 0:26:44, Enif
Procesando fila 4: 11/7/2018, 0:27:28, Enif
Procesando fila 5: 11/7/2018, 0:28:40, Enif
Procesando fila 6: 11/7/2018, 0:29:27, Enif
Procesando fila 7: 11/7/2018, 0:48:53, Sadalmelik
Procesando fila 8: 11/7/2018, 0:50:53, Sadalmelik
Procesando fila 9: 11/7/2018, 0:52:11, Sadalmelik
Procesando fila 10: 11/7/2018, 0:52:43, Sadalmelik
Procesando fila 11: 11/8/2018, 0:48:26, Fomalhaut
Procesando fila 12: 11/8/2018, 0:50:42, Fomalhaut
Procesando fila 13: 11/8/2018, 0:51:36, Fomalhaut
Procesando fila 14: 11/8/2018, 0:55:04, Fomalhaut
Procesando fila 15: 11/8/2018, 1:00:57, Altair
Procesando fila 16: 11/8/2018, 1:01:42, Altair
Procesando fila 17: 11/8/2018, 1:04:11, Altair
Procesando fila 18: 11/8/2018, 1:11:19, Enif
Procesando fila 19: 11/8/2018, 1:12:21, Enif
Procesando fila 20: 11/8/2018, 1:13:11, Enif
Proces